In [2]:
#Cragamos librerias
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
#Cargar desde un archivo .csv sin indice
df = pd.read_csv('Valencia_Datos_limpios.csv')
df.head(5)

,Unnamed: 0,listing_url,last_scraped,source,name,neighborhood_overview,picture_url,host_url,host_name,host_since,...,beds,minimum_nights,maximum_nights,availability_365,number_of_reviews,review_scores_rating,review_scores_cleanliness,review_scores_communication,review_scores_location,reviews_per_month
0,0,https://www.airbnb.com/rooms/48154,18/09/2024,city scrape,Precioso apartamento con wifi,Very close neighborhood to the city's old town...,https://a0.muscache.com/pictures/1109085/9c0d4...,https://www.airbnb.com/users/show/219476,Antonio,01/09/2010,...,2.0,3.0,25,149,22.1,4.61,4.63,4.79,4.51,1.01
1,1,https://www.airbnb.com/rooms/137143,18/09/2024,city scrape,PENTHOUSE ON FRONT LINE BEACH,Itâs a peaceful and friendly community.,https://a0.muscache.com/pictures/hosting/Hosti...,https://www.airbnb.com/users/show/670775,Cpi GestiÃÂ³n Encuentra Tu MansiÃÂ³n,06/06/2011,...,2.1,2.2,170,94,1.0,4.70,5.00,5.00,4.80,0.01
2,2,https://www.airbnb.com/rooms/149715,18/09/2024,city scrape,1900 Style Valencian Beach Home for 10px,"Surrounded by historic houses, authentic res...",https://a0.muscache.com/pictures/7507244a-87d8...,https://www.airbnb.com/users/show/5947,Susana Barbara,03/01/2009,...,2.1,2.0,1125,306,22.1,4.66,4.55,4.89,4.60,1.68
3,3,https://www.airbnb.com/rooms/165971,18/09/2024,city scrape,Ã¢ÂÂ Architectural touch! Ã¢ÂÂ,Just outside the old city centre (one block aw...,https://a0.muscache.com/pictures/10878830/ade6...,https://www.airbnb.com/users/show/791187,InÃÂ©s,07/07/2011,...,1.0,5.0,10,79,22.1,4.87,4.88,4.87,4.84,3.31
4,4,https://www.airbnb.com/rooms/169284,18/09/2024,city scrape,Apartment near downtown,It's a neighborhood with all amenities. 200 me...,https://a0.muscache.com/pictures/14393073/b549...,https://www.airbnb.com/users/show/806645,Maria Jesus,11/07/2011,...,1.0,2.0,1125,280,22.1,4.41,4.44,4.78,4.52,1.17


**Convertimos a numericos**

In [4]:
#df['host_response_time'] = df['host_response_time'].map({'a few days or more':1, 'within an hour':2,'within a few hours':3, 'within a day':4})
df['host_is_superhost'] = df['host_is_superhost'].map({'t': 1, 'f': 0})
df["host_response_rate"] = df["host_response_rate"] / 100
#df['room_type'] = df['room_type'].map({'Entire home/apt': 1,'Private room': 2,'Shared room': 3,'Hotel room': 4})
#df["host_acceptance_rate"] = df["host_acceptance_rate"].str.rstrip('%').astype(float) / 100
df['host_verifications'] = df['host_verifications'].map({"['email', 'phone', 'work_email']":1, "['email', 'phone']":2, "['phone']": 3,"['phone', 'work_email']": 4, "['email']": 5,"[]": 6,"['email', 'work_email']": 7,"['work_email']": 8})
df['host_has_profile_pic'] = df['host_has_profile_pic'].map({'t': 1, 'f': 0})
df['host_identity_verified'] = df['host_identity_verified'].map({'t': 1, 'f': 0})
df['has_availability'] = df['has_availability'].map({'t': 1, 'f': 0})
df['instant_bookable'] = df['instant_bookable'].map({'t': 1, 'f': 0})


In [5]:
df['property_type'] = df['property_type'].replace({
    'Entire villa': 1, 'Entire home': 2, 'Entire rental unit': 3, 'Entire loft': 4,
    'Private room': 5, 'Private room in rental unit': 6, 'Private room in home': 7,
    'Private room in condo': 8, 'Private room in guest suite': 9, 'Entire condo': 10,
    'Private room in serviced apartment': 11, 'Room in serviced apartment': 12,
    'Entire guest suite': 13, 'Private room in townhouse': 14, 'Room in boutique hotel': 15,
    'Private room in bed and breakfast': 16, 'Shared room in rental unit': 17,
    'Entire serviced apartment': 18, 'Tiny home': 19, 'Entire guesthouse': 20,
    'Private room in guesthouse': 21, 'Private room in loft': 22, 'Entire bungalow': 23,
    'Casa particular': 24, 'Shared room in loft': 25, 'Entire townhouse': 26,
    'Private room in tiny home': 27, 'Shared room in guesthouse': 28, 'Room in bed and breakfast': 29,
    'Shared room in hostel': 30, 'Private room in farm stay': 31, 'Shared room in home': 32,
    'Private room in villa': 33, 'Entire cabin': 34, 'Private room in casa particular': 35,
    'Room in hotel': 36, 'Private room in floor': 37, 'Private room in cabin': 38,
    'Shared room in guest suite': 39, 'Room in aparthotel': 40, 'Entire place': 41,
    'Private room in hostel': 42, 'Earthen home': 43, 'Entire hostel': 44,
    'Private room in earthen home': 45, 'Private room in dorm': 46, 'Private room in hut': 47,
    'Entire cottage': 48, 'Entire chalet': 49, 'Shared room in dorm': 50,
    'Entire in-law': 51, 'Holiday park': 52, 'Private room in dome': 53,
    'Shared room in condo': 54, 'Campsite': 55, 'Private room in vacation home': 56,
    'Room in casa particular': 57, 'Castle': 58, 'Private room in barn': 59,
    'Shared room in hotel': 60, 'Farm stay': 61, 'Shared room in tiny home': 62,
    'Hut': 63, 'Shared room in cabin': 64, 'Private room in cottage': 65,
    'Entire home/apt': 66, 'Room in hostel': 67, 'Shared room in serviced apartment': 68,
    'Entire vacation home': 69, 'Shared room in tent': 70, 'Shared room in bed and breakfast': 71,
    'Private room in shipping container': 72, 'Private room in nature lodge': 73,
    'Private room in tower': 74, 'Private room in lighthouse': 75,
    'Shared room in casa particular': 76, 'Shared room in townhouse': 77,
    'Shared room in farm stay': 78, 'Shared room in boutique hotel': 79,
    'Shipping container': 80, 'Private room in pension': 81, 'Private room in bungalow': 82,
    'Dome': 83, 'Tent': 84, 'Private room in chalet': 85, 'Private room in tent': 86,
    'Private room in castle': 87, 'Private room in houseboat': 88, 'Boat': 89,
    'Private room in resort': 90, 'Private room in minsu': 91, 'Tower': 92,
    'Private room in camper/rv': 93, 'Houseboat':94, 'Shared room in lighthouse':95,
    'Camper/RV':96
})


C:\Users\kidfr\AppData\Local\Temp\ipykernel_7040\2212853263.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['property_type'] = df['property_type'].replace({


In [6]:
df['property_type'].unique()

array([ 3,  2,  6,  4, 26,  1, 18, 10,  7, 41, 19, 89,  9,  8, 17, 54, 14,
       16, 15, 48, 42, 49, 67, 21, 35, 11, 23, 36, 69, 40, 13, 20,  5, 96,
       56, 22, 30, 24, 32, 93, 76, 94, 34, 58, 60, 25, 95])

**Reemplazar valores**

_host_acceptance_rate_

In [7]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['host_acceptance_rate'])
unico

array([0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99,
       1.  ])

In [8]:
df["host_acceptance_rate"] = df["host_acceptance_rate"]/ 100

In [9]:
#Convierto una variable a dicotómica
df['host_acceptance_rate'] = df['host_acceptance_rate'].apply(
    lambda x: 1 if isinstance(x, (int, float)) and x > 0.008 else (0 if isinstance(x, (int, float)) and x <= 0.008 else 'Sin respuesta')
)

df

,Unnamed: 0,listing_url,last_scraped,source,name,neighborhood_overview,picture_url,host_url,host_name,host_since,...,beds,minimum_nights,maximum_nights,availability_365,number_of_reviews,review_scores_rating,review_scores_cleanliness,review_scores_communication,review_scores_location,reviews_per_month
0,0,https://www.airbnb.com/rooms/48154,18/09/2024,city scrape,Precioso apartamento con wifi,Very close neighborhood to the city's old town...,https://a0.muscache.com/pictures/1109085/9c0d4...,https://www.airbnb.com/users/show/219476,Antonio,01/09/2010,...,2.0,3.0,25,149,22.1,4.61,4.63,4.79,4.51,1.01
1,1,https://www.airbnb.com/rooms/137143,18/09/2024,city scrape,PENTHOUSE ON FRONT LINE BEACH,Itâs a peaceful and friendly community.,https://a0.muscache.com/pictures/hosting/Hosti...,https://www.airbnb.com/users/show/670775,Cpi GestiÃÂ³n Encuentra Tu MansiÃÂ³n,06/06/2011,...,2.1,2.2,170,94,1.0,4.70,5.00,5.00,4.80,0.01
2,2,https://www.airbnb.com/rooms/149715,18/09/2024,city scrape,1900 Style Valencian Beach Home for 10px,"Surrounded by historic houses, authentic res...",https://a0.muscache.com/pictures/7507244a-87d8...,https://www.airbnb.com/users/show/5947,Susana Barbara,03/01/2009,...,2.1,2.0,1125,306,22.1,4.66,4.55,4.89,4.60,1.68
3,3,https://www.airbnb.com/rooms/165971,18/09/2024,city scrape,Ã¢ÂÂ Architectural touch! Ã¢ÂÂ,Just outside the old city centre (one block aw...,https://a0.muscache.com/pictures/10878830/ade6...,https://www.airbnb.com/users/show/791187,InÃÂ©s,07/07/2011,...,1.0,5.0,10,79,22.1,4.87,4.88,4.87,4.84,3.31
4,4,https://www.airbnb.com/rooms/169284,18/09/2024,city scrape,Apartment near downtown,It's a neighborhood with all amenities. 200 me...,https://a0.muscache.com/pictures/14393073/b549...,https://www.airbnb.com/users/show/806645,Maria Jesus,11/07/2011,...,1.0,2.0,1125,280,22.1,4.41,4.44,4.78,4.52,1.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9442,9442,https://www.airbnb.com/rooms/1247688630964779070,18/09/2024,city scrape,Preciosa habitaciÃÂ³n en Ruzafa en piso compa...,Itâs a peaceful and friendly community.,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/576254600,Hugo,07/05/2024,...,1.0,2.2,365,363,0.0,4.60,4.60,4.90,4.80,1.30
9443,9443,https://www.airbnb.com/rooms/1247692918887242627,19/09/2024,city scrape,Spirit of Mercat 1 - Chic Studio,Itâs a peaceful and friendly community.,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/569472969,Hugo Adrian,28/03/2024,...,1.0,2.2,335,323,0.0,4.60,4.60,4.90,4.80,1.30
9444,9444,https://www.airbnb.com/rooms/1247696223333878783,18/09/2024,city scrape,"The Nest, a high top floor",Itâs a peaceful and friendly community.,https://a0.muscache.com/pictures/hosting/Hosti...,https://www.airbnb.com/users/show/139048962,Cesar,07/07/2017,...,3.0,1.0,5,18,0.0,4.60,4.60,4.90,4.80,1.30
9445,9445,https://www.airbnb.com/rooms/1247844761890740750,18/09/2024,city scrape,Turia II,Itâs a peaceful and friendly community.,https://a0.muscache.com/pictures/prohost-api/H...,https://www.airbnb.com/users/show/438136382,Home Club,30/12/2021,...,1.0,1.0,365,283,0.0,4.60,4.60,4.90,4.80,1.30


_room_type_

In [10]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['room_type'])
unico

array(['Entire home/apt', 'Hotel room', 'Private room', 'Shared room'],
      dtype=object)

In [11]:
#Convierto una variable a dicotómica
df['room_type'] = df['room_type'].replace(["Shared room", "Private room"], 0)
df['room_type'] = df['room_type'].replace(["Entire home/apt", "Hotel room"], 1)
df

C:\Users\kidfr\AppData\Local\Temp\ipykernel_7040\508382164.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['room_type'] = df['room_type'].replace(["Entire home/apt", "Hotel room"], 1)


,Unnamed: 0,listing_url,last_scraped,source,name,neighborhood_overview,picture_url,host_url,host_name,host_since,...,beds,minimum_nights,maximum_nights,availability_365,number_of_reviews,review_scores_rating,review_scores_cleanliness,review_scores_communication,review_scores_location,reviews_per_month
0,0,https://www.airbnb.com/rooms/48154,18/09/2024,city scrape,Precioso apartamento con wifi,Very close neighborhood to the city's old town...,https://a0.muscache.com/pictures/1109085/9c0d4...,https://www.airbnb.com/users/show/219476,Antonio,01/09/2010,...,2.0,3.0,25,149,22.1,4.61,4.63,4.79,4.51,1.01
1,1,https://www.airbnb.com/rooms/137143,18/09/2024,city scrape,PENTHOUSE ON FRONT LINE BEACH,Itâs a peaceful and friendly community.,https://a0.muscache.com/pictures/hosting/Hosti...,https://www.airbnb.com/users/show/670775,Cpi GestiÃÂ³n Encuentra Tu MansiÃÂ³n,06/06/2011,...,2.1,2.2,170,94,1.0,4.70,5.00,5.00,4.80,0.01
2,2,https://www.airbnb.com/rooms/149715,18/09/2024,city scrape,1900 Style Valencian Beach Home for 10px,"Surrounded by historic houses, authentic res...",https://a0.muscache.com/pictures/7507244a-87d8...,https://www.airbnb.com/users/show/5947,Susana Barbara,03/01/2009,...,2.1,2.0,1125,306,22.1,4.66,4.55,4.89,4.60,1.68
3,3,https://www.airbnb.com/rooms/165971,18/09/2024,city scrape,Ã¢ÂÂ Architectural touch! Ã¢ÂÂ,Just outside the old city centre (one block aw...,https://a0.muscache.com/pictures/10878830/ade6...,https://www.airbnb.com/users/show/791187,InÃÂ©s,07/07/2011,...,1.0,5.0,10,79,22.1,4.87,4.88,4.87,4.84,3.31
4,4,https://www.airbnb.com/rooms/169284,18/09/2024,city scrape,Apartment near downtown,It's a neighborhood with all amenities. 200 me...,https://a0.muscache.com/pictures/14393073/b549...,https://www.airbnb.com/users/show/806645,Maria Jesus,11/07/2011,...,1.0,2.0,1125,280,22.1,4.41,4.44,4.78,4.52,1.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9442,9442,https://www.airbnb.com/rooms/1247688630964779070,18/09/2024,city scrape,Preciosa habitaciÃÂ³n en Ruzafa en piso compa...,Itâs a peaceful and friendly community.,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/576254600,Hugo,07/05/2024,...,1.0,2.2,365,363,0.0,4.60,4.60,4.90,4.80,1.30
9443,9443,https://www.airbnb.com/rooms/1247692918887242627,19/09/2024,city scrape,Spirit of Mercat 1 - Chic Studio,Itâs a peaceful and friendly community.,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/569472969,Hugo Adrian,28/03/2024,...,1.0,2.2,335,323,0.0,4.60,4.60,4.90,4.80,1.30
9444,9444,https://www.airbnb.com/rooms/1247696223333878783,18/09/2024,city scrape,"The Nest, a high top floor",Itâs a peaceful and friendly community.,https://a0.muscache.com/pictures/hosting/Hosti...,https://www.airbnb.com/users/show/139048962,Cesar,07/07/2017,...,3.0,1.0,5,18,0.0,4.60,4.60,4.90,4.80,1.30
9445,9445,https://www.airbnb.com/rooms/1247844761890740750,18/09/2024,city scrape,Turia II,Itâs a peaceful and friendly community.,https://a0.muscache.com/pictures/prohost-api/H...,https://www.airbnb.com/users/show/438136382,Home Club,30/12/2021,...,1.0,1.0,365,283,0.0,4.60,4.60,4.90,4.80,1.30


_availability_365_

In [12]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['availability_365'])
unico

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [13]:
#Convierto una variable a dicotómica
df['availability_365'] = df['availability_365'].apply(
    lambda x: 1 if x == 365 else (0 if isinstance(x, (int, float)) and x < 365 else 'Sin respuesta')
)

df

,Unnamed: 0,listing_url,last_scraped,source,name,neighborhood_overview,picture_url,host_url,host_name,host_since,...,beds,minimum_nights,maximum_nights,availability_365,number_of_reviews,review_scores_rating,review_scores_cleanliness,review_scores_communication,review_scores_location,reviews_per_month
0,0,https://www.airbnb.com/rooms/48154,18/09/2024,city scrape,Precioso apartamento con wifi,Very close neighborhood to the city's old town...,https://a0.muscache.com/pictures/1109085/9c0d4...,https://www.airbnb.com/users/show/219476,Antonio,01/09/2010,...,2.0,3.0,25,0,22.1,4.61,4.63,4.79,4.51,1.01
1,1,https://www.airbnb.com/rooms/137143,18/09/2024,city scrape,PENTHOUSE ON FRONT LINE BEACH,Itâs a peaceful and friendly community.,https://a0.muscache.com/pictures/hosting/Hosti...,https://www.airbnb.com/users/show/670775,Cpi GestiÃÂ³n Encuentra Tu MansiÃÂ³n,06/06/2011,...,2.1,2.2,170,0,1.0,4.70,5.00,5.00,4.80,0.01
2,2,https://www.airbnb.com/rooms/149715,18/09/2024,city scrape,1900 Style Valencian Beach Home for 10px,"Surrounded by historic houses, authentic res...",https://a0.muscache.com/pictures/7507244a-87d8...,https://www.airbnb.com/users/show/5947,Susana Barbara,03/01/2009,...,2.1,2.0,1125,0,22.1,4.66,4.55,4.89,4.60,1.68
3,3,https://www.airbnb.com/rooms/165971,18/09/2024,city scrape,Ã¢ÂÂ Architectural touch! Ã¢ÂÂ,Just outside the old city centre (one block aw...,https://a0.muscache.com/pictures/10878830/ade6...,https://www.airbnb.com/users/show/791187,InÃÂ©s,07/07/2011,...,1.0,5.0,10,0,22.1,4.87,4.88,4.87,4.84,3.31
4,4,https://www.airbnb.com/rooms/169284,18/09/2024,city scrape,Apartment near downtown,It's a neighborhood with all amenities. 200 me...,https://a0.muscache.com/pictures/14393073/b549...,https://www.airbnb.com/users/show/806645,Maria Jesus,11/07/2011,...,1.0,2.0,1125,0,22.1,4.41,4.44,4.78,4.52,1.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9442,9442,https://www.airbnb.com/rooms/1247688630964779070,18/09/2024,city scrape,Preciosa habitaciÃÂ³n en Ruzafa en piso compa...,Itâs a peaceful and friendly community.,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/576254600,Hugo,07/05/2024,...,1.0,2.2,365,0,0.0,4.60,4.60,4.90,4.80,1.30
9443,9443,https://www.airbnb.com/rooms/1247692918887242627,19/09/2024,city scrape,Spirit of Mercat 1 - Chic Studio,Itâs a peaceful and friendly community.,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/569472969,Hugo Adrian,28/03/2024,...,1.0,2.2,335,0,0.0,4.60,4.60,4.90,4.80,1.30
9444,9444,https://www.airbnb.com/rooms/1247696223333878783,18/09/2024,city scrape,"The Nest, a high top floor",Itâs a peaceful and friendly community.,https://a0.muscache.com/pictures/hosting/Hosti...,https://www.airbnb.com/users/show/139048962,Cesar,07/07/2017,...,3.0,1.0,5,0,0.0,4.60,4.60,4.90,4.80,1.30
9445,9445,https://www.airbnb.com/rooms/1247844761890740750,18/09/2024,city scrape,Turia II,Itâs a peaceful and friendly community.,https://a0.muscache.com/pictures/prohost-api/H...,https://www.airbnb.com/users/show/438136382,Home Club,30/12/2021,...,1.0,1.0,365,0,0.0,4.60,4.60,4.90,4.80,1.30


_host_response_time_

In [14]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['host_response_time'])
unico

array(['a few days or more', 'within a day', 'within a few hours',
       'within an hour'], dtype=object)

In [15]:
df['host_response_time']=df['host_response_time'].replace(["within a few hours", "within a day", "a few days or more"], 0)
df['host_response_time']=df['host_response_time'].replace(['within an hour'], 1)
df

C:\Users\kidfr\AppData\Local\Temp\ipykernel_7040\1826624729.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['host_response_time']=df['host_response_time'].replace(['within an hour'], 1)


,Unnamed: 0,listing_url,last_scraped,source,name,neighborhood_overview,picture_url,host_url,host_name,host_since,...,beds,minimum_nights,maximum_nights,availability_365,number_of_reviews,review_scores_rating,review_scores_cleanliness,review_scores_communication,review_scores_location,reviews_per_month
0,0,https://www.airbnb.com/rooms/48154,18/09/2024,city scrape,Precioso apartamento con wifi,Very close neighborhood to the city's old town...,https://a0.muscache.com/pictures/1109085/9c0d4...,https://www.airbnb.com/users/show/219476,Antonio,01/09/2010,...,2.0,3.0,25,0,22.1,4.61,4.63,4.79,4.51,1.01
1,1,https://www.airbnb.com/rooms/137143,18/09/2024,city scrape,PENTHOUSE ON FRONT LINE BEACH,Itâs a peaceful and friendly community.,https://a0.muscache.com/pictures/hosting/Hosti...,https://www.airbnb.com/users/show/670775,Cpi GestiÃÂ³n Encuentra Tu MansiÃÂ³n,06/06/2011,...,2.1,2.2,170,0,1.0,4.70,5.00,5.00,4.80,0.01
2,2,https://www.airbnb.com/rooms/149715,18/09/2024,city scrape,1900 Style Valencian Beach Home for 10px,"Surrounded by historic houses, authentic res...",https://a0.muscache.com/pictures/7507244a-87d8...,https://www.airbnb.com/users/show/5947,Susana Barbara,03/01/2009,...,2.1,2.0,1125,0,22.1,4.66,4.55,4.89,4.60,1.68
3,3,https://www.airbnb.com/rooms/165971,18/09/2024,city scrape,Ã¢ÂÂ Architectural touch! Ã¢ÂÂ,Just outside the old city centre (one block aw...,https://a0.muscache.com/pictures/10878830/ade6...,https://www.airbnb.com/users/show/791187,InÃÂ©s,07/07/2011,...,1.0,5.0,10,0,22.1,4.87,4.88,4.87,4.84,3.31
4,4,https://www.airbnb.com/rooms/169284,18/09/2024,city scrape,Apartment near downtown,It's a neighborhood with all amenities. 200 me...,https://a0.muscache.com/pictures/14393073/b549...,https://www.airbnb.com/users/show/806645,Maria Jesus,11/07/2011,...,1.0,2.0,1125,0,22.1,4.41,4.44,4.78,4.52,1.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9442,9442,https://www.airbnb.com/rooms/1247688630964779070,18/09/2024,city scrape,Preciosa habitaciÃÂ³n en Ruzafa en piso compa...,Itâs a peaceful and friendly community.,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/576254600,Hugo,07/05/2024,...,1.0,2.2,365,0,0.0,4.60,4.60,4.90,4.80,1.30
9443,9443,https://www.airbnb.com/rooms/1247692918887242627,19/09/2024,city scrape,Spirit of Mercat 1 - Chic Studio,Itâs a peaceful and friendly community.,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/569472969,Hugo Adrian,28/03/2024,...,1.0,2.2,335,0,0.0,4.60,4.60,4.90,4.80,1.30
9444,9444,https://www.airbnb.com/rooms/1247696223333878783,18/09/2024,city scrape,"The Nest, a high top floor",Itâs a peaceful and friendly community.,https://a0.muscache.com/pictures/hosting/Hosti...,https://www.airbnb.com/users/show/139048962,Cesar,07/07/2017,...,3.0,1.0,5,0,0.0,4.60,4.60,4.90,4.80,1.30
9445,9445,https://www.airbnb.com/rooms/1247844761890740750,18/09/2024,city scrape,Turia II,Itâs a peaceful and friendly community.,https://a0.muscache.com/pictures/prohost-api/H...,https://www.airbnb.com/users/show/438136382,Home Club,30/12/2021,...,1.0,1.0,365,0,0.0,4.60,4.60,4.90,4.80,1.30


_accommodates_

In [16]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['accommodates'])
unico

array([1. , 2. , 3. , 3.2, 4. , 5. , 6. , 7. ])

In [17]:
df['accommodates'] = df['accommodates'].replace([1, 2, 3], 0).replace([3.2, 4, 5, 6, 7], 1)
df

,Unnamed: 0,listing_url,last_scraped,source,name,neighborhood_overview,picture_url,host_url,host_name,host_since,...,beds,minimum_nights,maximum_nights,availability_365,number_of_reviews,review_scores_rating,review_scores_cleanliness,review_scores_communication,review_scores_location,reviews_per_month
0,0,https://www.airbnb.com/rooms/48154,18/09/2024,city scrape,Precioso apartamento con wifi,Very close neighborhood to the city's old town...,https://a0.muscache.com/pictures/1109085/9c0d4...,https://www.airbnb.com/users/show/219476,Antonio,01/09/2010,...,2.0,3.0,25,0,22.1,4.61,4.63,4.79,4.51,1.01
1,1,https://www.airbnb.com/rooms/137143,18/09/2024,city scrape,PENTHOUSE ON FRONT LINE BEACH,Itâs a peaceful and friendly community.,https://a0.muscache.com/pictures/hosting/Hosti...,https://www.airbnb.com/users/show/670775,Cpi GestiÃÂ³n Encuentra Tu MansiÃÂ³n,06/06/2011,...,2.1,2.2,170,0,1.0,4.70,5.00,5.00,4.80,0.01
2,2,https://www.airbnb.com/rooms/149715,18/09/2024,city scrape,1900 Style Valencian Beach Home for 10px,"Surrounded by historic houses, authentic res...",https://a0.muscache.com/pictures/7507244a-87d8...,https://www.airbnb.com/users/show/5947,Susana Barbara,03/01/2009,...,2.1,2.0,1125,0,22.1,4.66,4.55,4.89,4.60,1.68
3,3,https://www.airbnb.com/rooms/165971,18/09/2024,city scrape,Ã¢ÂÂ Architectural touch! Ã¢ÂÂ,Just outside the old city centre (one block aw...,https://a0.muscache.com/pictures/10878830/ade6...,https://www.airbnb.com/users/show/791187,InÃÂ©s,07/07/2011,...,1.0,5.0,10,0,22.1,4.87,4.88,4.87,4.84,3.31
4,4,https://www.airbnb.com/rooms/169284,18/09/2024,city scrape,Apartment near downtown,It's a neighborhood with all amenities. 200 me...,https://a0.muscache.com/pictures/14393073/b549...,https://www.airbnb.com/users/show/806645,Maria Jesus,11/07/2011,...,1.0,2.0,1125,0,22.1,4.41,4.44,4.78,4.52,1.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9442,9442,https://www.airbnb.com/rooms/1247688630964779070,18/09/2024,city scrape,Preciosa habitaciÃÂ³n en Ruzafa en piso compa...,Itâs a peaceful and friendly community.,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/576254600,Hugo,07/05/2024,...,1.0,2.2,365,0,0.0,4.60,4.60,4.90,4.80,1.30
9443,9443,https://www.airbnb.com/rooms/1247692918887242627,19/09/2024,city scrape,Spirit of Mercat 1 - Chic Studio,Itâs a peaceful and friendly community.,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/569472969,Hugo Adrian,28/03/2024,...,1.0,2.2,335,0,0.0,4.60,4.60,4.90,4.80,1.30
9444,9444,https://www.airbnb.com/rooms/1247696223333878783,18/09/2024,city scrape,"The Nest, a high top floor",Itâs a peaceful and friendly community.,https://a0.muscache.com/pictures/hosting/Hosti...,https://www.airbnb.com/users/show/139048962,Cesar,07/07/2017,...,3.0,1.0,5,0,0.0,4.60,4.60,4.90,4.80,1.30
9445,9445,https://www.airbnb.com/rooms/1247844761890740750,18/09/2024,city scrape,Turia II,Itâs a peaceful and friendly community.,https://a0.muscache.com/pictures/prohost-api/H...,https://www.airbnb.com/users/show/438136382,Home Club,30/12/2021,...,1.0,1.0,365,0,0.0,4.60,4.60,4.90,4.80,1.30


In [18]:
df['availability_365']

0       0
1       0
2       0
3       0
4       0
       ..
9442    0
9443    0
9444    0
9445    0
9446    0
Name: availability_365, Length: 9447, dtype: int64

**REGRESIÓN LOGÍSTICA**

_host_is_superhost_

In [19]:
#Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep=df[['host_acceptance_rate', 'host_response_rate', 'number_of_reviews']]
Var_Dep = df['host_is_superhost']

In [20]:
X= Vars_Indep
y= Var_Dep

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=None)

In [22]:
escalar =StandardScaler()

In [23]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [24]:
from sklearn.linear_model import LogisticRegression
algoritma = LogisticRegression()

In [25]:
algoritma.fit(X_train, y_train)

LogisticRegression()

In [26]:
y_pred =algoritma.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0], shape=(5669,))

In [27]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[3851  241]
 [1381  196]]


In [28]:
#Calcula la prescisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.448512585812357


In [29]:
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.71388251896278


In [30]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad =recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.9411045943304008


_host_identity_verified_

In [31]:
#Declaramos las variables dependientes e independientes para regresión logistica
Vars_Indep=df[['has_availability', 'host_response_rate', 'host_acceptance_rate']]
Var_Dep=df['host_identity_verified']

In [32]:
df['host_acceptance_rate'].dtype

dtype('int64')

In [33]:
X1= Vars_Indep
y1= Var_Dep

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.3, random_state=None)

In [35]:
escalar =StandardScaler()

In [36]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [37]:
from sklearn.linear_model import LogisticRegression
algoritma = LogisticRegression()

In [38]:
algoritma.fit(X_train, y_train)

LogisticRegression()

In [39]:
y_pred =algoritma.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 1], shape=(2835,))

In [40]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[   0   69]
 [   0 2766]]


In [41]:
#Calcula la prescisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.9756613756613757


In [42]:
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.9756613756613757


In [43]:
from sklearn.metrics import recall_score

sensibilidad =recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.0


_host_acceptance_rate_

In [44]:
Vars_Indep= df[["accommodates", "room_type","instant_bookable"]]
Var_Dep= df["host_acceptance_rate"]

In [45]:
X= Vars_Indep
y= Var_Dep

In [46]:
X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.3, random_state=None)

In [47]:
escalar= StandardScaler()

In [48]:
X_train= escalar.fit_transform(X_train)
X_test= escalar.fit_transform(X_test)

In [49]:
from sklearn.linear_model import LogisticRegression
algoritmo= LogisticRegression()

In [50]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [51]:
y_pred= algoritmo.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 1], shape=(2835,))

In [52]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusion: ")
print(matriz)

Matriz de confusion: 
[[   0   71]
 [   0 2764]]


In [53]:
from sklearn.metrics import precision_score

precision= precision_score(y_test, y_pred, average="binary", pos_label=1)
print("predicciones del modelo")
print(precision)

predicciones del modelo
0.9749559082892416


In [54]:
from sklearn.metrics import accuracy_score

exactitud= accuracy_score(y_test, y_pred)
print("Exactitud del modelo")
print(exactitud)

Exactitud del modelo
0.9749559082892416


In [55]:
from sklearn.metrics import recall_score

sensibilidad= recall_score(y_test, y_pred, average="binary", pos_label=0)
print("Sensibilidad del modelo: ")
print(sensibilidad)

Sensibilidad del modelo: 
0.0


_has_availability_

In [56]:
Vars_Indep=df[['host_identity_verified', 'host_response_rate', 'host_acceptance_rate']]
Var_Dep=df['has_availability']

In [57]:
df['host_identity_verified'].dtype

dtype('int64')

In [58]:
#Redefinimos las variables
X3 = Vars_Indep
y3 = Var_Dep

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X3, y3, test_size=0.2, random_state=None)

In [60]:
escalar =StandardScaler()

In [61]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [62]:
from sklearn.linear_model import LogisticRegression
algoritma = LogisticRegression()

In [63]:
algoritma.fit(X_train, y_train)

LogisticRegression()

In [64]:
y_pred =algoritma.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 1], shape=(1890,))

In [65]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[   0   43]
 [   0 1847]]


In [66]:
#Calcula la prescisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.9772486772486773


In [67]:
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.9772486772486773


In [68]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad =recall_score(y_test, y_pred, average="binary", pos_label= 0)
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.0


_instant_bookable_

In [69]:
Vars_Indep= df[["accommodates", "room_type"]]
Var_Dep= df["instant_bookable"]

In [70]:
X= Vars_Indep
y= Var_Dep

In [71]:
X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.3, random_state=None) 

In [72]:
escalar= StandardScaler()

In [73]:
X_train= escalar.fit_transform(X_train)
X_test= escalar.fit_transform(X_test)

In [74]:
from sklearn.linear_model import LogisticRegression
algoritmo= LogisticRegression()

In [75]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [76]:
y_pred= algoritmo.predict(X_test)
y_pred

array([1, 0, 0, ..., 1, 0, 0], shape=(2835,))

In [77]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusion: ")
print(matriz)

Matriz de confusion: 
[[1342  271]
 [ 844  378]]


In [78]:
from sklearn.metrics import precision_score

precision= precision_score(y_test, y_pred, average="binary", pos_label=1)
print("predicciones del modelo")
print(precision)

predicciones del modelo
0.5824345146379045


In [79]:
from sklearn.metrics import accuracy_score

exactitud= accuracy_score(y_test, y_pred)
print("Exactitud del modelo")
print(exactitud)

Exactitud del modelo
0.6067019400352733


In [80]:
from sklearn.metrics import recall_score

sensibilidad= recall_score(y_test, y_pred, average="binary", pos_label=0)
print("Sensibilidad del modelo: ")
print(sensibilidad)

Sensibilidad del modelo: 
0.8319900805951643


_room_type_

In [81]:
Vars_Indep= df[["accommodates", "room_type", "instant_bookable"]]
Var_Dep= df["room_type"]

In [82]:
X= Vars_Indep
y= Var_Dep

In [83]:
X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.3, random_state=None) 

In [84]:
escalar= StandardScaler()

In [85]:
X_train= escalar.fit_transform(X_train)
X_test= escalar.fit_transform(X_test)

In [86]:
from sklearn.linear_model import LogisticRegression
algoritmo= LogisticRegression()

In [87]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [88]:
y_pred= algoritmo.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 1], shape=(2835,))

In [89]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusion: ")
print(matriz)

Matriz de confusion: 
[[ 851    0]
 [   0 1984]]


In [90]:
from sklearn.metrics import precision_score

precision= precision_score(y_test, y_pred, average="binary", pos_label=1)
print("predicciones del modelo")
print(precision)

predicciones del modelo
1.0


In [91]:
from sklearn.metrics import accuracy_score

exactitud= accuracy_score(y_test, y_pred)
print("Exactitud del modelo")
print(exactitud)

Exactitud del modelo
1.0


In [92]:
from sklearn.metrics import recall_score

sensibilidad= recall_score(y_test, y_pred, average="binary", pos_label=0)
print("Sensibilidad del modelo: ")
print(sensibilidad)

Sensibilidad del modelo: 
1.0


_accommodates_

In [93]:
Vars_Indep= df[[ "room_type", "instant_bookable"]]
Var_Dep= df["accommodates"]

In [94]:
X= Vars_Indep
y= Var_Dep

In [95]:
X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.3, random_state=None) 

In [96]:
escalar= StandardScaler()

In [97]:
X_train= escalar.fit_transform(X_train)
X_test= escalar.fit_transform(X_test)

In [98]:
from sklearn.linear_model import LogisticRegression
algoritmo= LogisticRegression()

In [99]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [100]:
y_pred= algoritmo.predict(X_test)
y_pred

array([0., 1., 0., ..., 0., 0., 1.], shape=(2835,))

In [101]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusion: ")
print(matriz)

Matriz de confusion: 
[[ 800  626]
 [  36 1373]]


In [102]:
from sklearn.metrics import precision_score

precision= precision_score(y_test, y_pred, average="binary", pos_label=1)
print("predicciones del modelo")
print(precision)

predicciones del modelo
0.6868434217108554


In [103]:
from sklearn.metrics import accuracy_score

exactitud= accuracy_score(y_test, y_pred)
print("Exactitud del modelo")
print(exactitud)

Exactitud del modelo
0.7664902998236331


In [104]:
from sklearn.metrics import recall_score

sensibilidad= recall_score(y_test, y_pred, average="binary", pos_label=0)
print("Sensibilidad del modelo: ")
print(sensibilidad)

Sensibilidad del modelo: 
0.5610098176718092


_availability_365_

In [105]:
df["price"] = df["price"].str.replace("$", "", regex=False).str.replace(",", "").astype(float).astype(int)

In [106]:
#DEclaramos las variables dependientes e independientes para la regresión logística
Vars_Indep=df[['host_response_rate','price', 'review_scores_location', 'property_type', 'host_identity_verified']]
Var_Dep = df['availability_365']

In [107]:
df['review_scores_cleanliness'].dtype

dtype('float64')

In [108]:
#Redefinimos las variables
X = Vars_Indep
y = Var_Dep

In [109]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state = None)

In [110]:
#Se escalan todos los datos
escalar = StandardScaler()

In [111]:
#Para realizar el escalamineto de las variables "X" tanto de entrenamiento como de prueba, utilizaremos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [112]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [113]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [114]:
#REalizamos una prediccion
y_pred = algoritmo.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0], shape=(3779,))

In [115]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[3738    0]
 [  41    0]]


In [116]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.0


c:\Users\kidfr\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [117]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Excatitud del modelo:')
print(exactitud)

Excatitud del modelo:
0.9891505689335803


In [118]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
1.0


_host_response_time_

In [119]:
#DEclaramos las variables dependientes e independientes para la regresión logística
Vars_Indep=df[['host_verifications', 'host_total_listings_count', 'host_response_rate']]
Var_Dep = df['host_response_time']

In [120]:
df['host_total_listings_count'].dtype

dtype('float64')

In [121]:
#Redefinimos las variables
X = Vars_Indep
y = Var_Dep

In [122]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = None)

In [123]:
#Se escalan todos los datos
escalar = StandardScaler()

In [124]:
#Para realizar el escalamineto de las variables "X" tanto de entrenamiento como de prueba, utilizaremos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [125]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [126]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [127]:
#REalizamos una prediccion
y_pred = algoritmo.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 1], shape=(2835,))

In [128]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[   0  633]
 [   0 2202]]


In [129]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.7767195767195767


In [130]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Excatitud del modelo:')
print(exactitud)

Excatitud del modelo:
0.7767195767195767


In [131]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.0


_host_has_profile_pic_

In [132]:
#DEclaramos las variables dependientes e independientes para la regresión logística
Vars_Indep=df[['host_verifications', 'review_scores_rating', 'host_total_listings_count', 'host_is_superhost']]
Var_Dep = df['host_has_profile_pic']

In [133]:
df['review_scores_rating'].dtype

dtype('float64')

In [134]:
#Redefinimos las variables
X = Vars_Indep
y = Var_Dep

In [135]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = None)

In [136]:
#Se escalan todos los datos
escalar = StandardScaler()

In [137]:
#Para realizar el escalamineto de las variables "X" tanto de entrenamiento como de prueba, utilizaremos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [138]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [139]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [140]:
#REalizamos una prediccion
y_pred = algoritmo.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 1], shape=(2835,))

In [141]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[   0  142]
 [   2 2691]]


In [142]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.9498764560536533


In [143]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Excatitud del modelo:')
print(exactitud)

Excatitud del modelo:
0.9492063492063492


In [144]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.0
